Чтобы решить эту задачу, мы можем использовать различные методы машинного обучения, такие как логистическая регрессия, случайный лес или градиентный бустинг. Но прежде чем начать кодирование, нам нужно предварительно обработать данные, чтобы подготовить их для обучения модели.

Давайте разработаем общий план:

Загрузим данные и объединим их в соответствии с информацией о клиентах.
Подготовим признаки для обучения модели.
Обучим модель машинного обучения для предсказания оттока клиентов.
Оценим производительность модели.
Для начала нам нужно распаковать файл transactions.csv.zip и загрузить все данные. После этого мы сможем продолжить с предобработкой данных и построением модели.

In [1]:
!cd /content && wget https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/DataFusion2024/Data/clients.csv
!cd /content && wget https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/DataFusion2024/Data/train.csv
!cd /content && wget https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/DataFusion2024/Data/report_dates.csv
!cd /content && wget https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/DataFusion2024/Data/transactions.csv.zip

--2024-03-07 19:10:14--  https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/DataFusion2024/Data/clients.csv
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2457605 (2.3M) [text/csv]
Saving to: ‘clients.csv’

clients.csv         100%[===================>]   2.34M  1.78MB/s    in 1.3s    

2024-03-07 19:10:17 (1.78 MB/s) - ‘clients.csv’ saved [2457605/2457605]

--2024-03-07 19:10:17--  https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/DataFusion2024/Data/train.csv
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 754462 (737K) [text/csv]
Saving to: ‘train.csv’

t

In [65]:
import warnings
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [2]:
'''
from google.colab import drive

warnings.filterwarnings('ignore')
sns.set(style='darkgrid')
%matplotlib inline

# отображение_всех_строк_и_столбцов
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# подключение_к_диску
drive.mount('/content/drive')

pd.options.mode.chained_assignment = None
'''

Mounted at /content/drive


In [66]:
# Загрузка данных
clients = pd.read_csv("clients.csv")
train = pd.read_csv("train.csv")
report_dates = pd.read_csv("report_dates.csv")
transactions = pd.read_csv("transactions.csv.zip")

In [4]:
# Создание целевой переменной
# Предполагаем, что метка 1 означает уход клиента (отток), а 0 - остаток
# Преобразуем метку 0 в 1, так как в задаче Survival Analysis 1 обозначает событие, а 0 - цензурированные данные
#train['event_status'] = 1 - train['target']

In [67]:
# Объединение данных
data = pd.merge(train, clients, on='user_id', how='left')

In [68]:
# Преобразование даты отчета в формат даты
report_dates['report_dt'] = pd.to_datetime(report_dates['report_dt'])

In [69]:
# Предполагаем, что время до последней транзакции определено относительно последнего отчета
data['last_transaction_dt'] = report_dates['report_dt'].max() - pd.to_timedelta(data['time'], unit='D')

In [70]:
# Преобразование даты транзакции в формат даты
transactions['transaction_dttm'] = pd.to_datetime(transactions['transaction_dttm'])

In [71]:
# Рассчитаем общую сумму транзакций для каждого клиента за последние 6 месяцев
transactions_last_6_months = transactions[transactions['transaction_dttm'] >= report_dates['report_dt'].max() - pd.DateOffset(months=6)]
total_transaction_amt_last_6_months = transactions_last_6_months.groupby('user_id')['transaction_amt'].sum().reset_index()
total_transaction_amt_last_6_months.rename(columns={'transaction_amt': 'total_transaction_amt_last_6_months'}, inplace=True)

In [ ]:
'''
Группа 0 может представлять детей или лиц моложе 18 лет.
Группа 1 может представлять молодежь или лиц от 18 до 30 лет.
Группа 2 может представлять взрослых или лиц от 30 до 50 лет.
Группа 3 может представлять пожилых людей или лиц старше 50 лет.
'''

In [72]:
# Объединим данные с общей суммой транзакций в последние 6 месяцев с основными данными
data = pd.merge(data, total_transaction_amt_last_6_months, on='user_id', how='left')

In [73]:
data.dropna(inplace=True)            # удаление всех строк, содержащих пропущенные значения(NaN).

In [74]:
# Преобразование категориального признака 'employee_count_nm' с помощью One-Hot Encoding
data = pd.get_dummies(data, columns=['employee_count_nm'])

In [75]:
# Подготовим данные для обучения модели
X = data.drop(['user_id', 'target', 'time', 'last_transaction_dt'], axis=1)
y = data['target']

In [76]:
# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [77]:
# Обучим модель случайного леса
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [78]:
# Предскажем отток на тестовом наборе
y_pred = model.predict(X_test)

# Оценка производительности модели
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.9284611425630468
              precision    recall  f1-score   support

           0       0.96      0.97      0.96      1852
           1       0.11      0.08      0.09        91

    accuracy                           0.93      1943
   macro avg       0.53      0.52      0.53      1943
weighted avg       0.92      0.93      0.92      1943



In [79]:
# Предсказание оттока для конкретного клиента
churn_prediction = model.predict(X_test[2:3])

# Вывод результата предсказания
if churn_prediction == 1:
    print("Клиент уйдет в отток.")
else:
    print("Клиент останется.")


Клиент останется.


In [80]:
# Применение обученной модели к данным всех клиентов для предсказания оттока
predictions = model.predict(X_test)

# Индексы клиентов, которые покинут банк (прогноз оттока = 1)
churned_clients_indices = X_test.index[predictions == 1]

# Получение данных о клиентах, которые покинут банк
churned_clients_data = X_test.loc[churned_clients_indices]

# Вывод данных о клиентах, которые покинут банк
print(churned_clients_data.index)
print(f"{churned_clients_data.shape[0]} Клиента покинут банк.")

Int64Index([ 5129, 63152, 28975, 10130, 24480, 26373, 56178, 62531,  8340,
            18764, 13380,  9410, 19519, 58763, 35370, 13600, 21963, 36035,
             2195, 27207, 26792,  9114, 32422,  8828, 35323, 49307, 10673,
             5382, 40610, 26911, 22018, 11572, 39041, 14842,  2339,  1875,
            23091, 33134, 15686,  2345, 19800, 16630, 24270, 15697, 56929,
             7991, 19141, 18755, 49831, 62886,  5373,  6344, 14385, 40559,
            29900, 15393, 27569, 32429, 21414,  2350, 26703, 48662],
           dtype='int64')
62 Клиента покинут банк.


In [81]:
# Индексы клиентов, которые останутся в банке (прогноз оттока = 0)
churned_clients_indices = X_test.index[predictions == 0]

# Получение данных о клиентах, которые останутся в банке
churned_clients_data = X_test.loc[churned_clients_indices]

print(f"{churned_clients_data.shape[0]} Клиента останутся в банке.")


1881 Клиента останутся в банке.


In [82]:
# Загрузка тестовых данных
test_data = pd.read_csv("clients.csv")

In [83]:
# Преобразование категориального признака 'employee_count_nm' с помощью One-Hot Encoding
test_data = pd.get_dummies(test_data, columns=['employee_count_nm'])

In [84]:
test_data_mdf =  test_data.drop(['user_id'], axis=1)

In [87]:
test_data_mdf.insert(loc=3, column='total_transaction_amt_last_6_months', value=9543.386719)

In [88]:
predictions = model.predict(test_data_mdf)

In [92]:
# Создание DataFrame с предсказаниями
results_df = pd.DataFrame({'user_id': test_data['user_id'], 'predict': predictions})
results_df['predict'] = results_df['predict'].astype(float)
# Сохранение результатов в файл CSV
results_df.to_csv("predictions.csv", index=False)

print("Предсказания сохранены в файл predictions.csv")

Предсказания сохранены в файл predictions.csv


In [93]:
preds = pd.read_csv("predictions.csv")

In [91]:
# Define the variable 'predict'
predict = preds["predict"]

# Индексы клиентов, которые покинут банк (прогноз оттока = 1)
churned_clients_indices = preds.index[predict == 1]

# Получение данных о клиентах, которые покинут банк
churned_clients_data = preds.loc[churned_clients_indices]

# Вывод данных о клиентах, которые покинут банк
print(churned_clients_data.index)
print(f"{churned_clients_data.shape[0]} Клиента покинут банк.")

Int64Index([  634,  1229,  1423,  1990,  2130,  2328,  2356,  2444,  2469,
             3121,
            ...
            93904, 94200, 94258, 94450, 94538, 95104, 95178, 95322, 95609,
            95810],
           dtype='int64', length=761)
761 Клиента покинут банк.
